# Downsample DEM

In order to compare the data association with the Florian's implementation we need a dataset with groundtruth.
The DEM datasets have that but they were recorded at 10.000 Hz and have a size of more than 1 GB each.

Additionally they have to be filtered. 

Therefore we filter and resample the DEM datasets to 100 Hz each (same frequency as Florian used).

In [1]:
import numpy as np
import h5py

In [2]:
with h5py.File('groundtruthCylinders.mat', 'r') as file_:
    gt_x_y_vx_vy = file_.get('gt_x_y_vx_vy').value

In [6]:
print('Tracks: ', gt_x_y_vx_vy.shape[0])
print('Timesteps: ', gt_x_y_vx_vy.shape[1])
print('Data points per measurement (x,y,v_x, v_y): ', gt_x_y_vx_vy.shape[2])
gt_x_y_vx_vy.shape

Tracks:  4427
Timesteps:  19999
Data points per measurement (x,y,v_x, v_y):  4


(4427, 19999, 4)

## Filtern

Die Daten enthalten auch die Positionen der Partikel, wenn sie noch in der Schale liegen und nicht auf das Band geschüttet wurden.
Daher müssen wir filtern.

1. Beobachtungen in Bandgrenzen
2. Partikel, die das Band seitlich verlassen (das ist im Datensatz nicht nötig)

In [21]:
beltBordersX =[0.388, 0.788]
beltBordersY =[0, 0.18]

In [56]:
# Beobachtungen von Partikeln nur auf dem Band
gt_x_y_vx_vy[gt_x_y_vx_vy[:, :, 0] < beltBordersX[0]] = np.nan
gt_x_y_vx_vy[gt_x_y_vx_vy[:, :, 0] > beltBordersX[1]] = np.nan

In [59]:
# Keine Partikel, die das Band seitlich verlassen
assert not np.any(gt_x_y_vx_vy[:, :, 1] < beltBordersY[0])
assert not np.any(gt_x_y_vx_vy[:, :, 1] > beltBordersY[1])

# Downsamplen


In [63]:
old_hertz = 1000
new_hertz = 100
step_size = old_hertz // new_hertz

downsampled = gt_x_y_vx_vy[:, ::step_size, :2]
downsampled = downsampled.copy()
downsampled.shape

(4427, 2000, 2)

In [64]:
# Die Datei soll folgende Dimensionen haben: [Zeitschritte, 2 * AnzahlPartikel]
data = np.swapaxes(downsampled, 0, 1).reshape([downsampled.shape[1], -1])

In [65]:
np.savetxt("DEM_cylinder.csv", data, delimiter=',')